In [1]:
import time
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import svm
import ast

In [2]:
#Loading the self-supervised cxr-50x1-remedis-m model from the Aziz et al, 2022 paper
module = hub.load(R'C:\Users\tripl\Downloads\medical-ai-research-foundations-a-repository-of-medical-foundation-models-1.0.0\cxr-50x1-remedis-m')

In [3]:
#chunksize is set to 500 so that the data is processed in batches of 500
chunksize = 500
#The csv file containing the names and labels of the filtered data is loaded
df = pd.read_csv("Filterd_Data_14_Class_Chest X_Ray14", chunksize=chunksize)
#The directory of the chest X-ray14 images 
path = R"D:\Chest X-ray14 Dataset\images" 


#counter keeps track of the number of batches processed 
counter = 1

#The data is loaded in chunks, every chunk the data is fed into the self-supervised model to extract the feature representations
#These feature representations are stored in the X_train list with the accompanying encoded integer labels in the y_train list, 
#these are then saved to a npz file 
for chunk in df:
    #This dictionray will store the values of the batch and will be used to save the values to a npz file
    arrays_dict = {}
    X_train = []
    y_train = []
    for index, row in chunk.iterrows():
        #Images are loaded in RGB format and resized to (448,448) and converted to a float
        image = cv2.imread(path + "\\" + row["Image Index"], 1)
        resized_array = cv2.resize(image,(448, 448))
        resized_array = resized_array.astype(np.float32)
        resized_array = np.expand_dims(resized_array, axis=0)
        #Data is fed into the self-supervised mdoel
        embedding_of_image = module(resized_array)
        #The output feature representation from model is a (14,14,2048) tensor which is subsequently flattened to a (401408) 
        #tensor
        flattened_tensor = tf.reshape(embedding_of_image, [-1])
        array = flattened_tensor.numpy()
        arrays_dict[f'{row["Image Index"]}'] = array
        label = ast.literal_eval(row["Labels"])
        label = np.array(label)
        arrays_dict[f'{row["Image Index"]} Label'] = label
        #This is the path were the feature representations are saved
        np.savez(f'D:\\SSD downloads\\Processed Chest X-ray 14\\Updated Processed Chest X-rays {counter}.npz', **arrays_dict)
    counter += 1

The code executed in 64062.371975660324 seconds.
